<a href="https://colab.research.google.com/github/amitadhainje/MY_NOTES/blob/master/Spark_Transformations_and_Actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/srivatsan88/Mastering-Apache-Spark
#spark installation
!pip install pyspark

     |████████████████████████████████| 204.2MB 69kB/s 
     |████████████████████████████████| 204kB 41.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=1cedb7b122b095dfba24ba99a61618d5c3ff6cd7b9515232b6303247fd9254c9
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
!ls

sample_data  spark-3.0.1-bin-hadoop2.7	spark-3.0.1-bin-hadoop2.7.tgz


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
import sys, tempfile, urllib.request
from pyspark.sql.functions import *

BASE_DIR = '/tmp'
CORONA_DATA_FILE = os.path.join(BASE_DIR, 'corona_data.csv')

corona_data=urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv', CORONA_DATA_FILE)

In [7]:
BASE_DIR = '/tmp'
TWITTER_DATA_FILE = os.path.join(BASE_DIR, 'twitter_data.csv')
tweet_data=urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv', TWITTER_DATA_FILE)
!ls /tmp

blockmgr-8c707495-fddd-4988-a062-de0264725d5e
corona_data.csv
hsperfdata_root
spark-7ebdc0c6-941d-45d8-832e-72f77e2dcee4
spark-df9491c0-79ed-4e8d-bc60-48ec3a3817f0
twitter_data.csv


In [8]:
corona_df = spark.read.option("inferSchema", "true").csv("/tmp/corona_data.csv", header=True)
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [9]:
twitter_df = spark.read.option("inferSchema", "true").csv("/tmp/twitter_data.csv", header=True)
twitter_df.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

In [10]:
twitter_df.count()

1000

In [11]:
corona_df.count()

28143

In [12]:
twitter_df.filter('country="USA"').show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [13]:
twitter_df.filter('country="USA" and location like "%New%"').show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen| 

In [14]:
tw_df = twitter_df.filter('country="USA"')
tw_df.explain()

== Physical Plan ==
*(1) Project [_c0#110, geo#111, text#112, user#113, location#114, entities#115, sentiment#116, country#117]
+- *(1) Filter (isnotnull(country#117) AND (country#117 = USA))
   +- FileScan csv [_c0#110,geo#111,text#112,user#113,location#114,entities#115,sentiment#116,country#117] Batched: false, DataFilters: [isnotnull(country#117), (country#117 = USA)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,USA)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [15]:
tw_df = twitter_df.filter(col("location").startswith("N"))
tw_df.explain()

== Physical Plan ==
*(1) Project [_c0#110, geo#111, text#112, user#113, location#114, entities#115, sentiment#116, country#117]
+- *(1) Filter (isnotnull(location#114) AND StartsWith(location#114, N))
   +- FileScan csv [_c0#110,geo#111,text#112,user#113,location#114,entities#115,sentiment#116,country#117] Batched: false, DataFilters: [isnotnull(location#114), StartsWith(location#114, N)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(location), StringStartsWith(location,N)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [16]:
twitter_df.show(5)

+---+----+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|         location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|            Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|             -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|              NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          vic_gibson|          

In [17]:
# gives the first record
twitter_df.first()

Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria')

In [18]:
# first and tail functins gives the row objects
twitter_df.tail(5)


[Row(_c0=995, geo=None, text=" Thanks, Cindy. It's a safe bet to assume that I would do a better job of leading our nation's #coronavirus task force th…", user='makgrl', location='Hoosierland', entities="[('Cindy', 'PERSON'), '']", sentiment="{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'compound': 0.8271}", country='USA'),
 Row(_c0=996, geo=None, text=' Meanwhile In Australia Customers fight over toilet paper amid coronavirus panic#coronavirusindia #coronavirus #COVID19 #…', user='WebKarobar', location=None, entities="[('Australia', 'GPE'), '']", sentiment="{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.3818}", country=None),
 Row(_c0=997, geo=None, text=' More people died in the city of Chicago over the weekend (due to some dem’s corruption &amp; addiction to power over people) th…', user='freegalt', location=None, entities="[('Chicago', 'GPE'), ('the weekend', 'DATE')]", sentiment="{'neg': 0.15, 'neu': 0.85, 'pos': 0.0, 'compound': -0.5965}", country=None),
 Row(_c0=998, geo=No

**Narrow transformation** - In Narrow transformation, all the elements that are required to compute the records in single partition live in the single partition of parent RDD. A limited subset of partition is used to calculate the result. Narrow transformations are the result of map(), filter().

**Wide Transformation** -  In wide transformation, all the elements that are required to compute the records in the single partition may live in many partitions of parent RDD. The partition may live in many partitions of parent RDD. Wide transformations are the result of groupbyKey and reducebyKey.

<H3>Difference between flatmap and map</h3>

Both map() and flatMap() are used for transformations. The map() transformation takes in a function and applies it to each element in the RDD and the result of the function is a new value of each element in the resulting RDD. The flatMap() is used to produce multiple output elements for each input element.

Check following link for  more information and explanations - https://stackoverflow.com/questions/22350722/what-is-the-difference-between-map-and-flatmap-and-a-good-use-case-for-each

In [19]:
x = spark.sparkContext.parallelize([1,4,8])
y = x.flatMap(lambda x: (x, x*x))
print (x.collect())
print (y.collect())

[1, 4, 8]
[1, 1, 4, 16, 8, 64]


In [20]:
x = spark.sparkContext.parallelize([1,4,8])
y = x.map(lambda x: (x, x*x))
print (x.collect())
print (y.collect())

[1, 4, 8]
[(1, 1), (4, 16), (8, 64)]


In [21]:
twitter_df.select("text", "user").show(5)

+--------------------+--------------------+
|                text|                user|
+--------------------+--------------------+
|What is God sayin...|          petodinice|
|"BREAKING: ""this...| but i took the t...|
| #Coronavirus tes...|              cek422|
| Get ready for ma...|        InfectiousDz|
| The #coronavirus...|          vic_gibson|
+--------------------+--------------------+
only showing top 5 rows



In [22]:
twitter_df.rdd.map(lambda line: line.text.split(" ")).take(5)

[['What', 'is', 'God', 'saying', 'to', 'us', 'about', '#coronavirus', '?'],
 ['"BREAKING:', '""this', 'is', 'disappointing'],
 ['',
  '#Coronavirus',
  'testing',
  'must',
  'be',
  'made',
  'free',
  'to',
  'the',
  'public',
  'if',
  'we',
  'are',
  'going',
  'to',
  'understand',
  'the',
  'scope',
  'of',
  'this',
  'crisis.',
  'Anything',
  'le…'],
 ['',
  'Get',
  'ready',
  'for',
  'mass',
  'event',
  'crowd',
  'cancellations',
  'across',
  'the',
  'World',
  'starting',
  'this',
  'weekend:',
  'cricket',
  'in',
  '#Australia',
  'in',
  'empty',
  'st…'],
 ['',
  'The',
  '#coronavirus',
  'pandemic',
  'is',
  'revealing',
  'just',
  'how',
  'closely',
  'we',
  'are',
  'all',
  'bound',
  'together...[A',
  'thread]',
  '']]

In [23]:
twitter_df.rdd.flatMap(lambda line: line.text.split(" ")).take(20)

['What',
 'is',
 'God',
 'saying',
 'to',
 'us',
 'about',
 '#coronavirus',
 '?',
 '"BREAKING:',
 '""this',
 'is',
 'disappointing',
 '',
 '#Coronavirus',
 'testing',
 'must',
 'be',
 'made',
 'free']

In [24]:
corona_df.columns

['_c0',
 'State',
 'Country',
 'Lat',
 'Long',
 'Date',
 'Confirmed',
 'Death',
 'Recovered',
 'state_cleaned',
 'City']

In [25]:
corona_df.filter('Country="US"').sort(col('Date'), ascending=True).show()

+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|_c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
| 98|      Washington|     US|           47.4009|-121.4905|2020-01-22|        0|    0|        0|      Washington|      Washington|
| 99|        New York|     US|           42.1657| -74.9481|2020-01-22|        0|    0|        0|        New York|        New York|
|100|      California|     US|           36.1162|-119.6816|2020-01-22|        0|    0|        0|      California|      California|
|101|   Massachusetts|     US|           42.2302| -71.5301|2020-01-22|        0|    0|        0|   Massachusetts|   Massachusetts|
|102|Diamond Princess|     US|           35.4437|  139.638|2020-01-22|        0|   

In [26]:
corona_df.filter('Country="US"').orderBy(col('Date'), ascending=False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

**Sort** and **orderBy** are quite expensive operations. Data has to get moved between multiple executors nodes. By using **sortWithinPartitions**, we are restricting the data movements. 

In [27]:
corona_df.filter('Country="US"').sortWithinPartitions([col('Date'), col('Confirmed')], ascending=False).show()

+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|  _c0|         State|Country|    Lat|     Long|      Date|Confirmed|Death|Recovered| state_cleaned|          City|
+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|27765|      New York|     US|42.1657| -74.9481|2020-03-20|     8310|   42|        0|      New York|      New York|
|27764|    Washington|     US|47.4009|-121.4905|2020-03-20|     1524|   83|        0|    Washington|    Washington|
|27766|    California|     US|36.1162|-119.6816|2020-03-20|     1177|   23|        0|    California|    California|
|27773|    New Jersey|     US|40.2989|  -74.521|2020-03-20|      890|   11|        0|    New Jersey|    New Jersey|
|27776|      Illinois|     US|40.3495| -88.9861|2020-03-20|      585|    5|        0|      Illinois|      Illinois|
|27772|       Florida|     US|27.7663| -81.6868|2020-03-20|      563|   

In [28]:
corona_df.describe().show()

+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|              Long|      Date|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|  count|            28143|    19116|      28143|             28143|             28143|     28143|             28143|             28143|             28143|        28143|      14573|
|   mean|          14071.0|     null|       null|30.965553459118834|-34.57031257861667|      null|161.88245744945456| 5.494368048893153| 60.17290267562094|         null|       null|
| stddev|8124.328649186959|     null|       null|19.365472826597646| 80.78375872452575|   

In [29]:
corona_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Death: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [30]:
corona_df.filter('Confirmed > 10000').sort(col('Confirmed')).show()

+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|  _c0| State|Country|    Lat|    Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|24005|  null|   Iran|   32.0|    53.0|2020-03-12|    10075|  429|     2959|       Tehran|null|
|22912|  null|  Italy|   43.0|    12.0|2020-03-10|    10149|  631|      724|         Rome|null|
|27346|France| France|46.2276|  2.2137|2020-03-19|    10871|  243|       12|       France|null|
| 5401| Hubei|  China|30.9756|112.2707|2020-02-02|    11177|  350|      295|        Hubei|null|
|24482|  null|   Iran|   32.0|    53.0|2020-03-13|    11364|  514|     2959|       Tehran|null|
|26253|  null|  Spain|   40.0|    -4.0|2020-03-17|    11748|  533|     1028|       Toledo|null|
|26723|  null|Germany|   51.0|     9.0|2020-03-18|    12327|   28|      105|       Berlin|null|
|23866|  null|  Italy|   43.0|    12.0|2

In [31]:
# Here , "0.9" specifies the level of approximation that we want. More the level of approximation , the execution will need more time.
# Here our dataset is small and hence the level of approximation = 0.9 does not matter
corona_df.filter('Confirmed > 10000').approxQuantile('Confirmed', [0.25,0.5,0.75,0.9,0.95], 0.9)

[10075.0, 10075.0, 10075.0, 10075.0, 67800.0]

In [32]:
corona_df.agg({'Date':"max"}).collect()

[Row(max(Date)='2020-03-20')]

In [33]:
corona_df.agg({'Date':"max", "confirmed":"max"}).collect()

[Row(max(confirmed)=67800, max(Date)='2020-03-20')]

In [34]:
max_date = corona_df.agg({'Date':"max"})
max_date.show()

+----------+
| max(Date)|
+----------+
|2020-03-20|
+----------+



In [35]:
import pyspark.sql.functions as F
corona_df.groupBy("Country","State_cleaned").agg(F.max("Date")).show()

+--------------+--------------------+----------+
|       Country|       State_cleaned| max(Date)|
+--------------+--------------------+----------+
|      Cameroon|             Yaounde|2020-03-20|
|         China|             Qinghai|2020-03-20|
|        Cyprus|             Nicosia|2020-03-20|
|            US|            Michigan|2020-03-20|
|      Portugal|              Lisbon|2020-03-20|
|            US|            Colorado|2020-03-20|
|United Kingdom|      Cayman Islands|2020-03-20|
|         China|              Hainan|2020-03-20|
|            US|            Missouri|2020-03-20|
|     Australia|Australian Capita...|2020-03-20|
|            US|                Guam|2020-03-20|
|        France|             Reunion|2020-03-20|
|      Colombia|        Cundinamarca|2020-03-20|
|          Cuba|              Havana|2020-03-20|
|     Mauritius|          Port Louis|2020-03-20|
|       Ukraine|                Kiev|2020-03-20|
|         Benin|          Porto-Novo|2020-03-20|
|   Switzerland|    

In [36]:
corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max('Date').alias("Date")), on=['Country', 'State_cleaned', 'Date'], how='inner').show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [37]:
corona_df.join(corona_df.groupBy("Country","State_cleaned").agg(F.max("Date").alias("Date")),on=['Country', 'State_cleaned','Date'],how="inner").sort("Confirmed",ascending=False).show(10)

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|    City|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|         China|           Hubei|2020-03-20|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|    null|
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|    null|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|    null|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|    null|
|          Iran|          Tehran|2020-03-20|27821|          null|   32.0|               53.0|    

In [38]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

ws=Window().partitionBy("Country","State_cleaned").orderBy(col("Date").desc())

corona_df.withColumn("row_num", row_number().over(ws)).show()

+-----+-----+--------+------------------+-------+----------+---------+-----+---------+-------------+----+-------+
|  _c0|State| Country|               Lat|   Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|row_num|
+-----+-----+--------+------------------+-------+----------+---------+-----+---------+-------------+----+-------+
|27745| null|Cameroon|3.8480000000000003|11.5021|2020-03-20|       20|    0|        0|      Yaounde|null|      1|
|27268| null|Cameroon|3.8480000000000003|11.5021|2020-03-19|       13|    0|        0|      Yaounde|null|      2|
|26791| null|Cameroon|3.8480000000000003|11.5021|2020-03-18|       10|    0|        0|      Yaounde|null|      3|
|26314| null|Cameroon|3.8480000000000003|11.5021|2020-03-17|       10|    0|        0|      Yaounde|null|      4|
|25837| null|Cameroon|3.8480000000000003|11.5021|2020-03-16|        4|    0|        0|      Yaounde|null|      5|
|25360| null|Cameroon|3.8480000000000003|11.5021|2020-03-15|        2|    0|        0|  

In [39]:
corona_df.withColumn("row_num", row_number().over(ws)).where(col("row_num")==1).show()

+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|  _c0|               State|       Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|    City|row_num|
+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|27745|                null|      Cameroon|3.8480000000000003|  11.5021|2020-03-20|       20|    0|        0|             Yaounde|    null|      1|
|27859|             Qinghai|         China|           35.7452|  95.9956|2020-03-20|       18|    0|       18|             Qinghai|    null|      1|
|27762|                null|        Cyprus|           35.1264|  33.4299|2020-03-20|       67|    0|        0|             Nicosia|    null|      1|
|27812|            Michigan|            US|           43.3266| -84.5361|2020-03-20|      552|    3|        0|   

In [40]:
corona_max_df=corona_df.join(corona_df.groupBy("Country","State_cleaned").agg(F.max("Date").alias("Date")),on=['Country', 'State_cleaned','Date'],how="inner")

In [41]:
corona_max_df.show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [42]:
#Columnar view of time series data
corona_df.groupby("Country").pivot("Date").agg(F.sum("Confirmed")).show()

+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|    Country|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|2020-02-19|2020-

In [43]:

corona_df.filter("Country=='US'").crosstab("State","Date").show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|         State_Date|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|

In [44]:
corona_max_df.groupBy("Country").agg({'Confirmed':'sum'}).show()

+-----------+--------------+
|    Country|sum(Confirmed)|
+-----------+--------------+
|       Chad|             1|
|   Paraguay|            13|
|     Russia|           253|
|    Senegal|            38|
|     Sweden|          1639|
| Cabo Verde|             1|
|     Guyana|             7|
|Philippines|           230|
|   Djibouti|             1|
|  Singapore|           385|
|   Malaysia|          1030|
|       Fiji|             1|
|     Turkey|           359|
|       Iraq|           208|
|    Germany|         19848|
|   Cambodia|            51|
|Afghanistan|            24|
|     Jordan|            85|
|   Maldives|            13|
|     Rwanda|            17|
+-----------+--------------+
only showing top 20 rows



In [45]:
corona_max_df.groupBy("Country").agg({'Confirmed':'sum','Recovered':'sum','Death':'sum'}).orderBy("sum(Confirmed)", ascending=False).show()

+--------------+--------------+----------+--------------+
|       Country|sum(Recovered)|sum(Death)|sum(Confirmed)|
+--------------+--------------+----------+--------------+
|         China|         71266|      3253|         81250|
|         Italy|          4440|      4032|         47021|
|         Spain|          1588|      1043|         20410|
|       Germany|           180|        67|         19848|
|          Iran|          6745|      1433|         19644|
|            US|             0|       244|         19100|
|        France|            12|       450|         12726|
|  Korea, South|          1540|        94|          8652|
|   Switzerland|            15|        54|          5294|
|United Kingdom|            67|       178|          4014|
|   Netherlands|             2|       107|          3003|
|       Austria|             9|         6|          2388|
|       Belgium|             1|        37|          2257|
|        Norway|             1|         7|          1914|
|        Swede

In [46]:
corona_df.filter("Country=='Italy'").sort("Date",ascending=False).show()

+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|  _c0|State|Country| Lat|Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|27682| null|  Italy|43.0|12.0|2020-03-20|    47021| 4032|     4440|         Rome|null|
|27205| null|  Italy|43.0|12.0|2020-03-19|    41035| 3405|     4440|         Rome|null|
|26728| null|  Italy|43.0|12.0|2020-03-18|    35713| 2978|     4025|         Rome|null|
|26251| null|  Italy|43.0|12.0|2020-03-17|    31506| 2503|     2941|         Rome|null|
|25774| null|  Italy|43.0|12.0|2020-03-16|    27980| 2158|     2749|         Rome|null|
|25297| null|  Italy|43.0|12.0|2020-03-15|    24747| 1809|     2335|         Rome|null|
|24820| null|  Italy|43.0|12.0|2020-03-14|    21157| 1441|     1966|         Rome|null|
|24343| null|  Italy|43.0|12.0|2020-03-13|    17660| 1266|     1439|         Rome|null|
|23866| null|  Italy|43.0|12.0|2

In [47]:
corona_max_df.withColumn("Active",corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death).sort("Active",ascending=False).show()

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|Active|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null| 38549|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|      null| 19601|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null| 17779|
|        France|          France|2020-03-20|27823|        France|46.2276|             2.2137|    12612|  450|       12|      null| 12150|
|          Iran|          Tehran|2

In [48]:
corona_max_df=corona_max_df.withColumn("Active",corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death)

In [49]:
corona_max_df.show(10)

+---------+----------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+------+
|  Country|   state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|Active|
+---------+----------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+------+
| Thailand|         Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|   279|
|    Japan|         Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|   739|
|Singapore|       Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|   261|
|    Nepal|       Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|     0|
| Malaysia|         Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     1030|    3|       87|null|   940|
|   Canada|British Columbia|2020-03-20|2

In [50]:
corona_max_df.groupBy("Country").sum("Active").orderBy("sum(Active)",ascending=False).show()

+--------------+-----------+
|       Country|sum(Active)|
+--------------+-----------+
|         Italy|      38549|
|       Germany|      19601|
|            US|      18856|
|         Spain|      17779|
|        France|      12264|
|          Iran|      11466|
|  Korea, South|       7018|
|         China|       6731|
|   Switzerland|       5225|
|United Kingdom|       3769|
|   Netherlands|       2894|
|       Austria|       2373|
|       Belgium|       2219|
|        Norway|       1906|
|        Sweden|       1607|
|       Denmark|       1327|
|      Portugal|       1009|
|      Malaysia|        940|
|        Canada|        922|
|       Czechia|        829|
+--------------+-----------+
only showing top 20 rows

